In [2]:
library("ShortRead")

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: BiocParallel

Loading required package: Biostrings

Loading required package: S4Vectors



In [57]:
BASE_PATH_NR1 = "/data/samples/AIRR-Seq/OURS/S3987Nr1"
BASE_PATH_NR2 = "/data/samples/AIRR-Seq/OURS/S3987Nr2"


SAMPLES = list(
    list(base_path=BASE_PATH_NR1, name="S3987Nr1-PBMC_heavy"),
    list(base_path=BASE_PATH_NR1, name="S3987Nr1-PBMC_light"),
    list(base_path=BASE_PATH_NR1, name="S3987Nr1-RA1_heavy"),
    list(base_path=BASE_PATH_NR1, name="S3987Nr1-RA1_light")
#     list(base_path=BASE_PATH_NR2, name="S3987Nr2-PBMC_heavy"),
#     list(base_path=BASE_PATH_NR2, name="S3987Nr2-PBMC_light"),
#     list(base_path=BASE_PATH_NR2, name="S3987Nr2-RAMOS_heavy"),
#     list(base_path=BASE_PATH_NR2, name="S3987Nr2-RA1_light")
)

In [47]:
# Helpers
get_path_sample <- function(sample_name, base_path = BASE_PATH) {
    return(paste0(base_path, "/", sample_name))
}

get_path_fastq <- function(sample_name, type = "raw", base_path = BASE_PATH) {
    
    if (type == "raw") {
        return(
            paste0(
                get_path_sample(sample_name, base_path),
                "/", sample_name, ".1.fastq"
            )
        )
    } else if (type == "primer_pass") {
        suffix = ".1_primers-pass.fastq"
    } else if (type == "pair_pass"){
        suffix = ".1_primers-pass_pair-pass.fastq"
    } else if (type == "under2") {
        suffix = "_under-2.fastq"
    } else if (type == "atleast2") {
        suffix = "_atleast-2.fastq"
    } else {
        stop(paste0("Unsupported fastq file type: ", type))
    }
    
    return(
        paste0(
            get_path_sample(sample_name, base_path),
            "/", "presto",
            "/", sample_name, suffix
        )
    )
}

get_path_igblast <- function(sample_name, base_path = BASE_PATH) {
    return(
        paste0(
            get_path_sample(sample_name, base_path), 
            "/", "changeo_igblast", 
            "/", sample_name, "_db-pass_with_translation.tsv"
        )
    )
}

get_path_clones <- function(sample_name, clone_file = "groups", base_path = BASE_PATH) {
    
    if (clone_file == "groups") {
        suffix = "_vjl_groups.tsv"
    } else if (clone_file == "clones") {
        suffix = "_with_clones.tsv"
    } else {
        stop(paste0("Unknown clone_file type: ", clone_file))
    }
    
    return(
        paste0(
            get_path_sample(sample_name, base_path),
            "/", "clones", 
            "/", sample_name, suffix
        )
    )
}

read_tsv <- function(filepath) {
    return(read.csv(filepath, sep='\t'))
}

$base_path
[1] "/data/samples/AIRR-Seq/OURS/S3987Nr1"

$name
[1] "S3987Nr1-PBMC_heavy"

In [50]:
sample = SAMPLE_NAMES[[1]]

igblast <- read_tsv(
    get_path_igblast(sample$name, sample$base_path)
)

clone_groups <- read_tsv(
    get_path_clones(sample$name, "groups", sample$base_path)
)

clones <- read_tsv(
    get_path_clones(sample$name, "clones", sample$base_path)
)

print("IgBlast");colnames(igblast);head(igblast);
print("Clone groups");colnames(clone_groups);head(clone_groups);
print("Clones");colnames(clones);head(clones)

[1] "IgBlast"


[1] "sequence_id"        "sequence"           "rev_comp"          
 [4] "productive"         "v_call"             "d_call"            
 [7] "j_call"             "sequence_alignment" "germline_alignment"
[10] "junction"           "junction_aa"        "v_cigar"           
[13] "d_cigar"            "j_cigar"            "stop_codon"        
[16] "vj_in_frame"        "locus"              "junction_length"   
[19] "np1_length"         "np2_length"         "v_sequence_start"  
[22] "v_sequence_end"     "v_germline_start"   "v_germline_end"    
[25] "d_sequence_start"   "d_sequence_end"     "d_germline_start"  
[28] "d_germline_end"     "j_sequence_start"   "j_sequence_end"    
[31] "j_germline_start"   "j_germline_end"     "v_score"           
[34] "v_identity"         "v_support"          "d_score"           
[37] "d_identity"         "d_support"          "j_score"           
[40] "j_identity"         "j_support"          "fwr1"              
[43] "fwr2"               "fwr3"               "fwr4"              
[46] "cdr1"               "cdr2"               "cdr3"              
[49] "duplicate_count"    "cprimer"            "vprimer"           
[52] "cdr3_aa_length"     "cdr3_aa_gravy"      "cdr3_aa_bulk"      
[55] "cdr3_aa_aliphatic"  "cdr3_aa_polarity"   "cdr3_aa_charge"    
[58] "cdr3_aa_basic"      "cdr3_aa_acidic"     "cdr3_aa_aromatic"  
[61] "Translation"

,sequence_id,sequence,rev_comp,productive,v_call,d_call,j_call,sequence_alignment,germline_alignment,junction,...,cdr3_aa_length,cdr3_aa_gravy,cdr3_aa_bulk,cdr3_aa_aliphatic,cdr3_aa_polarity,cdr3_aa_charge,cdr3_aa_basic,cdr3_aa_acidic,cdr3_aa_aromatic,Translation
,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,M03234:246:000000000-DCH35:1:1101:11631:21712,CTGCAAGACCTCTGGAGACACCTTCAGCACCCACACTATCAGTTGGGTGCGACAGGCCCCTGGACAAGGACTTGAGTGGATGGGAAGGATCATCCCTACTCTTGGTATTGCAAACTACGCTCAGAAGTTCCAGGACAGAGTCACGATCATCGCGGACAAATCCACGGGCACAGCCTCTGTGGAACTGAGCAGCCTGAGATCCGACGACACGGCCATTTATTACTGTGCGAAAAATCATGATAGTAGTCCCGGAGTGTTTCAAGTCTGGGGCCAGGGCACCCTAGTCACCGTCTCCTCAGCATCCTCGNNNNNNNNNNNNNNNNNNNNNNNNNNN,FALSE,TRUE,IGHV1-69*02,IGHD3-22*01,IGHJ1*01,.................................................................CTGCAAGACCTCTGGAGACACCTTC............AGCACCCACACTATCAGTTGGGTGCGACAGGCCCCTGGACAAGGACTTGAGTGGATGGGAAGGATCATCCCTACT......CTTGGTATTGCAAACTACGCTCAGAAGTTCCAG...GACAGAGTCACGATCATCGCGGACAAATCCACGGGCACAGCCTCTGTGGAACTGAGCAGCCTGAGATCCGACGACACGGCCATTTATTACTGTGCGAAAAATCATGATAGTAGTCCCGGAGTGTTTCAAGTCTGGGGCCAGGGCACCCTAGTCACCGTCTCCTCAG,CAGGTCCAGCTGGTGCAATCTGGGGCT...GAGGTGAAGAAGCCTGGGTCCTCGGTGAAGGTCTCCTGCAAGGCTTCTGGAGGCACCTTC............AGCAGCTATACTATCAGCTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGAAGGATCATCCCTATC......CTTGGTATAGCAAACTACGCACAGAAGTTCCAG...GGCAGAGTCACGATTACCGCGGACAAATCCACGAGCACAGCCTACATGGAGCTGAGCAGCCTGAGATCTGAGGACACGGCCGTGTATTACTGTGCGANNNNNNATGATAGTAGTNNNNNNNNNNNNNNNNNCTGGGGCCAGGGCACCCTGGTCACCGTCTCCTCAG,TGTGCGAAAAATCATGATAGTAGTCCCGGAGTGTTTCAAGTCTGG,...,15,-0.4400000,14.51267,0.4533333,8.546667,0.03814639,0.13333333,0.06666667,0.2000000,XCKTSGDTFSTHTISWVRQAPGQGLEWMGRIIPTLGIANYAQKFQDRVTIIADKSTGTASVELSSLRSDDTAIYYCAKNHDSSPGVFQVWGQGTLVTVSS
2,M03234:246:000000000-DCH35:1:1101:5073:17951,ACCTGCACTGTCTCTGGTTACTCCATCAGCAGTGGTTACTACTGGGGCTGGATCCGGCAGCCCCCAGGGAGGGGGCTGGAGTGGATTGGGAATATCTATCATAGTGAGAGCCCCTACTACAACCCGTCCCTCAAGGATCGAGTCACCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGACCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGAGAGTCACTAGGACTGGAACTCCCCACGTCAGTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATNNNNNNNNNNNNNNNNNNNNNNNNNNN,FALSE,TRUE,IGHV4-38-2*02,IGHD1-7*01,IGHJ4*02,...............................................................ACCTGCACTGTCTCTGGTTACTCCATCAGC.........AGTGGTTACTACTGGGGCTGGATCCGGCAGCCCCCAGGGAGGGGGCTGGAGTGGATTGGGAATATCTATCATAGT.........GAGAGCCCCTACTACAACCCGTCCCTCAAG...GATCGAGTCACCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGACCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGAGAGTCACTAGGACTGGAACTCCCCACGTCAGTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTTACTCCATCAGC.........AGTGGTTACTACTGGGGCTGGATCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGAGTATCTATCATAGT.........GGGAGCACCTACTACAACCCGTCCCTCAAG...AGTCGAGTCACCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGAGAGNNNNNNNNACTGGAACTNNNNNNNNNNNTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,TGTGCGAGAGTCACTAGGACTGGAACTCCCCACGTCAGTGACTACTGG,...,16,-0.6312500,14.95875,0.4250000,8.337500,1.03653717,0.18750000,0.06250000,0.1875000,TCTVSGYSISSGYYWGWIRQPPGRGLEWIGNIYHSESPYYNPSLKDRVTISVDTSKNQFSLKLTSVTAADTAVYYCARVTRTGTPHVSDYWGQGTLVTVSS
3,M03234:246:000000000-DCH35:1:1101:13373:14079,CTGCAAGGCTTCTGGTTACACCTTTACGCGCTATGCTCTCATCTGGGTGCGACAGGCCCCTGGACAAGGTCTTGAGTGGATGGGGATGATCGGCACTTACAATGGTCATCCAACTTATGCACAGCACTTCCACGACAGAGTCACCATGGCCGCAGACACATCCACGAACACAGCCTACATGGAGCTGCGGACCCTGAGATCTGACGACACGGCCGTCTATTACTGTGCGACCGCCGGATATAGATCCAATGGGACACTTTTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCGGCATCCCCGNNNNNNNNNNNNNNNNNNNNNNNNNNN,FALSE,TRUE,IGHV1-18*01,"IGHD5-12*01,IGHD5/OR15-5a*01,IGHD5/OR15-5b*01",IGHJ4*02,.................................................................CTGCAAGGCTTCTGGTTACACCTTT............ACGCGCTATGCTCTCATCTGGGTGCGACAGGCCCCTGGACAAGGTCTTGAGTGGATGGGGATGATCGGCACTTAC......AATGGTCATCCAACTTATGCACAGCACTTCCAC...GACAGAGTCACCATGGCCGCAGACACATCCACGAACACAG

[1] "Clone groups"


[1] "vjl_group"       "sequence_count"  "v_call"          "j_call"         
[5] "junction_length" "clone_count"     "clone_id"

,vjl_group,sequence_count,v_call,j_call,junction_length,clone_count,clone_id
,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>
1,1,35,IGHV1-18*01,IGHJ1*01,48,1,222
2,2,10,IGHV1-18*01,IGHJ1*01,54,1,402
3,3,2,IGHV1-2*02,IGHJ6*02,42,1,577
4,4,7,IGHV1-2*02,IGHJ6*02,45,3,"791,792,793"
5,5,1,IGHV1-2*02,IGHJ6*02,48,1,1053
6,6,18,IGHV1-2*02,IGHJ6*02,51,1,1252


[1] "Clones"


[1] "sequence_id"        "sequence"           "rev_comp"          
 [4] "productive"         "v_call"             "d_call"            
 [7] "j_call"             "sequence_alignment" "germline_alignment"
[10] "junction"           "junction_aa"        "v_cigar"           
[13] "d_cigar"            "j_cigar"            "stop_codon"        
[16] "vj_in_frame"        "locus"              "junction_length"   
[19] "np1_length"         "np2_length"         "v_sequence_start"  
[22] "v_sequence_end"     "v_germline_start"   "v_germline_end"    
[25] "d_sequence_start"   "d_sequence_end"     "d_germline_start"  
[28] "d_germline_end"     "j_sequence_start"   "j_sequence_end"    
[31] "j_germline_start"   "j_germline_end"     "v_score"           
[34] "v_identity"         "v_support"          "d_score"           
[37] "d_identity"         "d_support"          "j_score"           
[40] "j_identity"         "j_support"          "fwr1"              
[43] "fwr2"               "fwr3"               "fwr4"              
[46] "cdr1"               "cdr2"               "cdr3"              
[49] "duplicate_count"    "cprimer"            "vprimer"           
[52] "cdr3_aa_length"     "cdr3_aa_gravy"      "cdr3_aa_bulk"      
[55] "cdr3_aa_aliphatic"  "cdr3_aa_polarity"   "cdr3_aa_charge"    
[58] "cdr3_aa_basic"      "cdr3_aa_acidic"     "cdr3_aa_aromatic"  
[61] "Translation"        "clone_id"

,sequence_id,sequence,rev_comp,productive,v_call,d_call,j_call,sequence_alignment,germline_alignment,junction,...,cdr3_aa_gravy,cdr3_aa_bulk,cdr3_aa_aliphatic,cdr3_aa_polarity,cdr3_aa_charge,cdr3_aa_basic,cdr3_aa_acidic,cdr3_aa_aromatic,Translation,clone_id
,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>
1,M03234:246:000000000-DCH35:1:1102:17704:18331,ACCTGCACTGTCTCTGGTTACTCCATCAGCAGTGGTTACTACTGGGGCTGGGTCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGCAGTATCTATCATAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTCTCCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGAGAGGAAAAAGTGACATAGCAGTGGCTGGTACAGGTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATNNNNNNNNNNNNNNNNNNNNNNNNNNN,FALSE,TRUE,IGHV4-38-2*02,IGHD6-19*01,IGHJ4*02,...............................................................ACCTGCACTGTCTCTGGTTACTCCATCAGC.........AGTGGTTACTACTGGGGCTGGGTCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGCAGTATCTATCATAGT.........GGGAGCACCTACTACAACCCGTCCCTCAAG...AGTCGAGTCTCCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGAGAGGAAAAAGTGACATAGCAGTGGCTGGTACAGGTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTTACTCCATCAGC.........AGTGGTTACTACTGGGGCTGGATCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGAGTATCTATCATAGT.........GGGAGCACCTACTACAACCCGTCCCTCAAG...AGTCGAGTCACCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGAGAGNNNNNNNNNNNATAGCAGTGGCTGGTACNNNTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,TGTGCGAGAGGAAAAAGTGACATAGCAGTGGCTGGTACAGGTGACTACTGG,...,-0.2176471,12.90706,0.5764706,8.535294,-0.07535248,0.1176471,0.1176471,0.1176471,TCTVSGYSISSGYYWGWVRQPPGKGLEWIGSIYHSGSTYYNPSLKSRVSISVDTSKNQFSLKLSSVTAADTAVYYCARGKSDIAVAGTGDYWGQGTLVTVSS,1
2,M03234:246:000000000-DCH35:1:1102:6373:14395,ACCTGTGCTGTCTCTGGTTACTCCATCAGCAGTGGTTACTACTGGGGCTGGGTCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGCAGTATCTATCATAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTCTCCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGAGAGGAAAAAGTGACATAGCAGTGGCTGGTACAGGTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATNNNNNNNNNNNNNNNNNNNNNNNNNNN,FALSE,TRUE,IGHV4-38-2*01,IGHD6-19*01,IGHJ4*02,...............................................................ACCTGTGCTGTCTCTGGTTACTCCATCAGC.........AGTGGTTACTACTGGGGCTGGGTCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGCAGTATCTATCATAGT.........GGGAGCACCTACTACAACCCGTCCCTCAAG...AGTCGAGTCTCCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGAGAGGAAAAAGTGACATAGCAGTGGCTGGTACAGGTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,CAGGTGCAGCTGCAGGAGTCGGGCCCA...GGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCGCTGTCTCTGGTTACTCCATCAGC.........AGTGGTTACTACTGGGGCTGGATCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGAGTATCTATCATAGT.........GGGAGCACCTACTACAACCCGTCCCTCAAG...AGTCGAGTCACCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGAGANNNNNNNNNNNNATAGCAGTGGCTGGTACNNNTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,TGTGCGAGAGGAAAAAGTGACATAGCAGTGGCTGGTACAGGTGACTACTGG,...,-0.2176471,12.90706,0.5764706,8.535294,-0.07535248,0.1176471,0.1176471,0.1176471,TCAVSGYSISSGYYWGWVRQPPGKGLEWIGSIYHSGSTYYNPSLKSRVSISVDTSKNQFSLKLSSVTAADTAVYYCARGKSDIAVAGTGDYWGQGTLVTVSS,1
3,M03234:246:000000000-DCH35:1:1101:24675:14708,ACCTGCGCTGTCTCTGGTTACTCCATCAGCAGTGGTTACTACTGGGGCTGGGTCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGCAGTATCTATCATAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTCTCCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGAGAGGAAAAAGTGACATAGCAGTGGCTGGTACAGGTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATNNNNNNNNNNNNNNNNNNNNNNNNNNN,FALSE,TRUE,IGHV4-38-2*01,IGHD6-19*01,IGHJ4*02,...............................................................ACCTGCGCTGTCTCTGGTTACTCCATCAGC.........AGTGGTTACTACTGGGGCTGGGTCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGCAGTATCTATCATAGT.........GGGAGCACCTACTACAACCCGTCCCTCAAG...AGTCGAGTCTCCATATCAGTAGACACGTCCAAG

### Characterize sequencing results for each sample

In [6]:
count_reads_fastq <- function(fastq_path) {
    return(
        ShortRead::countFastq(fastq_path)$records
    )
}

In [58]:
res <- sapply(
    SAMPLES,
    function(sample) {
        counts = cbind(
            sample$name,
            count_reads_fastq(get_path_fastq(sample$name, type = "raw", sample$base_path)),
            count_reads_fastq(get_path_fastq(sample$name, type = "primer_pass", sample$base_path)),
            count_reads_fastq(get_path_fastq(sample$name, type = "pair_pass", sample$base_path)),
            count_reads_fastq(get_path_fastq(sample$name, type = "under2", sample$base_path)),
            count_reads_fastq(get_path_fastq(sample$name, type = "atleast2", sample$base_path))
        )
        return(counts)
    }
)

res <- data.frame(t(res))
colnames(res) = c("sample_name", "raw", "primer_pass", "pair_pass", "under2", "atleast2")
head(res)

,sample_name,raw,primer_pass,pair_pass,under2,atleast2
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,S3987Nr1-PBMC_heavy,293081,293081,293081,127350,14008
2,S3987Nr1-PBMC_light,264454,264454,264454,108203,13384
3,S3987Nr1-RA1_heavy,160950,160950,160950,51691,7026
4,S3987Nr1-RA1_light,69991,69991,69991,25974,3791


### Abundancy and diversity comparisons among samples